In [1]:

import argparse
import os
import numpy as np
# import pandas as pd
from nsd_access import NSDAccess
import scipy.io

from config import NSD_ROOT_DIR, DATA_ROOT_DIR
from tqdm import tqdm


subj = 'subj01'
atlasname = 'streams'

proxy_list = ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']
for proxy in proxy_list:
    os.environ[proxy] = 'http://58.34.83.134:31280'

nsda = NSDAccess(NSD_ROOT_DIR)
nsd_expdesign = scipy.io.loadmat(os.path.join(NSD_ROOT_DIR, 'nsddata/experiments/nsd/nsd_expdesign.mat'))

# for downlaod annotations
nsda.read_image_coco_info([0])

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


[{'image_id': 532481,
  'id': 541125,
  'caption': "A person kitesurfing over the waves of the ocean's shore."},
 {'image_id': 532481,
  'id': 542259,
  'caption': 'a kite surfer is doing a flying trick over some water'},
 {'image_id': 532481,
  'id': 547713,
  'caption': 'A man is flying up in the air and having fun. '},
 {'image_id': 532481,
  'id': 554427,
  'caption': 'A guy is waterboarding in the ocean on a windy day.'},
 {'image_id': 532481,
  'id': 558036,
  'caption': 'A person kite boarding in rough seas near the shoreline.'}]

In [2]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

size = 425
def calc(x, len, h, l, r):
    if (len <= 0):
        return 0
    l = h * l
    r = h * (1 - r)
    y = x + len
    return max(0, (min(y, r) - max(x, l)) / len)

def resize_bbox(bbox, cropBox, h, w):
    ratio = calc(bbox[0], bbox[2], w, cropBox[2], cropBox[3]) * calc(bbox[1], bbox[3], h, cropBox[0], cropBox[1])

    bbox[1] -= h * cropBox[0]
    h *= (1 - cropBox[0] - cropBox[1])

    bbox[0] -= w * cropBox[2]
    w = w * (1 - cropBox[2] - cropBox[3])

    bbox[0] *= size / w
    bbox[2] *= size / w

    bbox[1] *= size / h
    bbox[3] *= size / h

    # Boundary processing
    bbox[0] = max(0, bbox[0])
    bbox[1] = max(0, bbox[1])
    bbox[2] = min(size - bbox[0], bbox[2])
    bbox[3] = min(size - bbox[1], bbox[3])

    sratio = (bbox[2] * bbox[3]) / (size * size)

    return bbox, ratio, sratio


In [3]:

class_labels = [i for i in range(0, 200)]
for x in nsda.get_coco_cats_list():
    class_labels[x['id']] = x['name']

print(f"In total {len(class_labels)}")

loading annotations into memory...
Done (t=0.44s)
creating index...
index created!
In total 200


In [4]:

st = 0
en = 73000

id_list = list(range(st, en))
info_list = nsda.get_image_info(id_list)
info_Img_list = nsda.read_image_coco_info_Img(id_list, info_type = 'instances')
annotations_list = nsda.read_image_coco_info(id_list, info_type='instances')


loading annotations into memory...
Done (t=13.12s)
creating index...
index created!
loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
loading annotations into memory...
Done (t=11.26s)
creating index...
index created!
loading annotations into memory...
Done (t=1.71s)
creating index...
index created!


In [5]:
cls_dic = {}
with open('cls_choosen.txt', "r") as f:
    for line in f.readlines():
        name = ' '.join(line.strip().split(' ')[1:])
        cls_dic[name] = 1
print(cls_dic)
        

{'person': 1, 'chair': 1, 'dining table': 1, 'car': 1, 'bowl': 1, 'cup': 1, 'bottle': 1, 'umbrella': 1, 'truck': 1, 'giraffe': 1, 'motorcycle': 1, 'bench': 1, 'dog': 1, 'horse': 1, 'elephant': 1, 'train': 1, 'bus': 1, 'cat': 1, 'pizza': 1, 'boat': 1, 'airplane': 1, 'toilet': 1, 'sheep': 1, 'cow': 1, 'zebra': 1, 'surfboard': 1, 'donut': 1, 'couch': 1, 'cake': 1, 'sink': 1, 'bird': 1, 'bed': 1, 'suitcase': 1, 'tv': 1, 'laptop': 1, 'teddy bear': 1, 'skis': 1, 'vase': 1, 'sandwich': 1, 'oven': 1, 'clock': 1, 'kite': 1, 'tie': 1, 'refrigerator': 1, 'hot dog': 1, 'keyboard': 1, 'cell phone': 1, 'bear': 1, 'fire hydrant': 1, 'stop sign': 1}


In [6]:
n = 73000
show = False
filter_ratio = 0.01
filter_cls = False
repeat_time = 1

In [10]:
st = 0
en = st + n

id_list = id_list[st:en]
info_list = info_list[st:en]
info_Img_list = info_Img_list[st:en]
annotations_list = annotations_list[st:en]

outputs = nsda.get_coco_dataset().copy()
outputs['images'] = []
outputs['annotations'] = []
tot_annotation = 0

cnt = np.zeros(200)

for i, (info, info_Img, annotations) in enumerate(tqdm(list(zip(info_list, info_Img_list, annotations_list)))):
    info_Img_new = info_Img[0].copy()
    info_Img_new['id'] = i
    info_Img_new['width'] = size
    info_Img_new['height'] = size
    info_Img_new['file_name'] = f'{i:06}.png'
    outputs['images'].append(info_Img_new)

    cropBox = info['cropBox']
    cropBox = eval(cropBox)
    h, w = info_Img[0]['height'], info_Img[0]['width']

    if (show):
        img = nsda.read_images(i, show=False)
        # Draw the image
        fig, ax = plt.subplots()
        ax.imshow(img)

    # Draw the annotations with class labels
    for annotation in annotations:
        # print(annotation)
        bbox = annotation['bbox'].copy()
        bbox, ratio, sratio = resize_bbox(bbox, cropBox, h, w)

        if ratio < 0.1:
            continue

        if sratio < filter_ratio:
            continue
        # print(sratio)

        if filter_cls == True and class_labels[annotation['category_id']] not in cls_dic:
            continue

        def non_linear_map(x):
            return int(round(5 - np.log1p(x * (np.exp(4) - 1))))
        if (repeat_time > 1):
            r = non_linear_map(sratio)
        else:
            r = 1
        
        for j in range(r):
            annotation_new = annotation.copy()
            annotation_new['image_id'] = i
            annotation_new['bbox'] = bbox
            annotation_new['id'] = tot_annotation
            tot_annotation += 1
            outputs['annotations'].append(annotation_new)

            cnt[annotation['category_id']] += 1


        if (show):
            category_id = annotation['category_id']
            class_label = class_labels[category_id]  # Adjust index to match class_labels list
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(bbox[0], bbox[1], class_label, color='r')

    if (show):
        # Show the image with annotations
        plt.show()

import json

name = f'instances_{st}_{en}_{filter_ratio}'
if (filter_cls):
    name += '_filter_cls'
if (repeat_time > 1):
    name += f'_rt{repeat_time}'

outputs_path = os.path.join(DATA_ROOT_DIR, f'{name}.json')
print(outputs_path)

with open(outputs_path, "w") as f:
    json.dump(outputs, f)
    

loading annotations into memory...
Done (t=2.97s)
creating index...
index created!


100%|██████████| 73000/73000 [00:03<00:00, 19706.91it/s]


/mnt/workspace/maxinzhu/denghan/nsd_processed_data/instances_0_73000_0.01.json


In [11]:
a = list(range(200))
a = sorted(a, key=lambda x: -cnt[x])
for i in range(200):
    print(cnt[a[i]], class_labels[a[i]])

77330.0 person
11008.0 chair
7270.0 car
7201.0 dining table
4925.0 bowl
4283.0 cup
4050.0 bottle
3596.0 umbrella
3557.0 truck
3442.0 broccoli
3427.0 banana
3330.0 bench
3317.0 motorcycle
3305.0 book
3216.0 giraffe
2889.0 horse
2863.0 dog
2851.0 elephant
2845.0 bus
2821.0 train
2781.0 boat
2760.0 pizza
2755.0 cat
2717.0 couch
2698.0 sheep
2690.0 airplane
2677.0 cow
2676.0 toilet
2664.0 potted plant
2638.0 zebra
2637.0 surfboard
2633.0 donut
2602.0 sink
2589.0 cake
2568.0 carrot
2497.0 bird
2497.0 handbag
2419.0 bed
2418.0 tv
2401.0 suitcase
2158.0 bicycle
2158.0 laptop
2140.0 backpack
2086.0 teddy bear
2014.0 skis
1997.0 tennis racket
1936.0 oven
1902.0 sandwich
1902.0 vase
1864.0 orange
1845.0 clock
1831.0 kite
1829.0 skateboard
1735.0 apple
1681.0 wine glass
1601.0 knife
1580.0 tie
1492.0 fork
1399.0 spoon
1386.0 refrigerator
1221.0 traffic light
1210.0 hot dog
1150.0 keyboard
1128.0 cell phone
1033.0 baseball bat
995.0 bear
915.0 snowboard
889.0 fire hydrant
840.0 stop sign
804.0 mic